# Packing problems
Solution guide to the case study "ProRobotics: Robotic process automation of repetitive tasks"
This Jupyter notebook contains the following models
- Model for the bin packing problem
- Model for the 1D cutting stock problem
- Model for the robotic process automation: Model is given, but solution is available in separate Jupyter notebook
- The 2D cutting stock problem: Exercise and solution available in MS Excel from website


In [1]:
import pandas as pd
import numpy as np
import math
import pulp as p

# The bin packing problem
PARAMETERS<br>
- $n$ : Number of items (index $i$)<br>
- $w_i$ : Weight of each item<br>
- $c$: Capacity of each bin (the index $j$ is used for bins, but since each bin has the same capacity, $c$ is used and not $c_j$)<br>
- $ub$: Upper bound on the number of bins (found by the first-fit decreasing heuristic)<br>

DECISION VARIABLES<br>
- $x_{i,j}$: 1, if item $i$ is packed in bin $j$, 0 otherwise <br>
- $y_j$: 1, if bin $j$ is used in the<br>

MODEL <br>
\begin{align*}
\mathrm{Minimise} \sum_{j=1}^{ub}y_j &&
\end{align*}
\begin{align}
\mathrm{s.t.}
& \sum_{j=1}^{ub} x_{i,j} = 1 				&& i = 1, \ldots, n & \label{BPP_1bin}\\
& \sum_{1=1}^{n} w_i \times x_{i,j} = c \times y_j 		&& j = 1, \ldots, ub\label{BPP_capacity}\\
& x_{i,j} \in \{0,1\}								&& i = 1, \ldots, n; j = 1, \ldots, ub\label{BPP_binaryX}\\
& y_j \in \{0,1\}									&& j = 1, \ldots, ub\label{BPP_binaryY}
\end{align}

In [2]:
# Example data
#number_items = 9 
#w = [5, 3, 3, 3, 2, 2, 2, 2, 2]
number_items = 8
w = [5, 4, 3, 3, 3, 2, 2, 2] # first fit says 4, optimum says 3
capacity = c = 8

In [3]:
# This is the code for the first-fit-decreasing heuristic of the bin packing problem
def heur_FFD(c, w): 
    n = len(w)
    order = sorted([i for i in range(n)], key=lambda i: w[i], reverse=True) # sort by decreasing weights
    bin_for_item = [-1 for i in range(n)]
    bin_space = []
    for i in order:
        for j in range(len(bin_space)): # pack in the first bin in which the item fits
            if w[i] <= bin_space[j]:
                bin_for_item[i] = j
                bin_space[j] -= w[i]
                break
        if bin_for_item[i] < 0: # if no bin can accomodate the item, open a new bin
            j = len(bin_space)
            bin_for_item[i] = j
            bin_space.append(c - w[i])
    n_bins = len(bin_space)
    return n_bins, bin_for_item

In [4]:
# Find a heuristic solution on the number of bins (upper bound ub) (is called again in the MODEL_BPP)
UB, bin_for_item = heur_FFD(c, w)
print("UB on the number of bins used = ",UB)
binList = [i for i in range(UB)]

UB on the number of bins used =  4


In [5]:
def MODEL_BPP(TimeLimit,showModel,):
    
    # Find a heuristic solution on the number of bins (upper bound ub)
    UB, bin_for_item = heur_FFD(c, w)
    
    number = len(w)
    itemList = [i for i in range(number)]
    binList = [i for i in range(UB)]
    
    # Create a LP Maximization problem
    Lp_prob = p.LpProblem('csp', p.LpMinimize) 

    # Create problem Variables 
    X_vars = p.LpVariable.dicts("X",(itemList,binList),lowBound=0,upBound=1,cat="Integer")
    Y_vars = p.LpVariable.dicts("Y",binList,lowBound=0,upBound=1,cat="Integer")

    # Objective Function
    Lp_prob += p.lpSum(Y_vars[j] for j in binList)

    # Constraints
    for i in itemList:
        Lp_prob += p.lpSum([X_vars[i][j] for j in binList]) == 1, 'Bin('+str(i)+')'

    for j in binList:
        Lp_prob += p.lpSum([w[i] * X_vars[i][j] for i in itemList]) <= c * Y_vars[j], 'Capacity('+str(j)+')'

    # Solve the problem
    status = Lp_prob.solve(p.PULP_CBC_CMD(msg=False,maxSeconds=TimeLimit))

    # Show the model
    if showModel: 
        print(Lp_prob)

    # Show the solution
    print('The status of the solution:',p.LpStatus[status])
    print('The objective value =',p.value(Lp_prob.objective))
    
    # Print the solution
    OutputXList = [[X_vars[i][j].varValue for j in binList] for i in itemList]
    print("X =",OutputXList)

    OutputYList = [Y_vars[j].varValue for j in binList]
    print("Y =",OutputYList)

    # Print the solution in words
    for i in itemList:
        for j in binList:
            if OutputXList[i][j] > 0:
                print("Item", i, "is packed in bin ", j)

In [6]:
MODEL_BPP(60,False)

The status of the solution: Optimal
The objective value = 3.0
X = [[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]]
Y = [1.0, 1.0, 1.0, 0.0]
Item 0 is packed in bin  1
Item 1 is packed in bin  0
Item 2 is packed in bin  2
Item 3 is packed in bin  2
Item 4 is packed in bin  1
Item 5 is packed in bin  2
Item 6 is packed in bin  0
Item 7 is packed in bin  0


/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


# The 1D cutting stock problem

The 1D cutting stock problem is identical to the bin packing problem (and therefore, we'll use the code of the first-fit-decreasing heuristic to get an upper bound on the number of stocks)

PARAMETERS<br>
- $L$: Stock lengths <br>
- $n$: Number of item types (index $i$) <br>
- $l_i$: Order lengths of item $i$ <br>
- $d_i$ demand number of item $i$ <br>

DECISION VARIABLES<br>
- $x_{i,j}$: Number of items of type $i$ cut from stock $j$ <br>
- $ t_j$: Length of each leftover of stock $j$ (i.e. not used for cutting items)<br>
- $y_j$: 1, if stock $j$ is used for cutting items, 0, otherwise<br>

MODEL<br>
\begin{align*}
\mathrm{Minimise} \sum_{j=1}^{ub}t_j &&
\end{align*}
\begin{align}
\mathrm{s.t.}
& \sum_{j=1}^{ub} x_{i,j} = d_i 						&& i = 1, \ldots, n & \label{1DCSP_Demand}\\
& \sum_{1=1}^{n} l_i \times x_{i,j} + t_j = L \times y_j 		&& j = 1, \ldots, ub \label{1DCSP_Waste}\\
& x_{i,j} \in \mathbb{Z}^{0+}												&& i = 1, \ldots, n, j = 1, \ldots, ub \label{1DCSP_X}\\
& t_{j} \in \mathbb{Z}^{0+}											&& j = 1, \ldots, ub \label{1DCSP_T} \\
& y_j \in \{0,1\}												&& j = 1, \ldots, ub \label{1DCSP_Y}
\end{align}

In [7]:
# Example data instance with no waste (change the length of the last item into 1 (instead of 2) and you'll have waste)
number = 3 #number of items
length = [5, 3, 2] 
demand = [1, 3, 5]
stockLength = L = 8

number = 4 #number of items
length = [5, 4, 3, 2] 
demand = [1, 1, 3, 3]
stockLength = L = 8

itemList = [i for i in range(number)]

In [8]:
# Find a heuristic solution (upper bound) on the number of stocks used (using the heuristic for the BPP)

# The first-fit-decreasing heuristic for the bin packing problem can also be used for the 1D cutting stock problem as follows:
# split_items - the number of items n is equal to number of items x demand d
# w_list - each item has a weigth w equal to its length l
# capacity - each stock has a capacity c equal to L

split_items = sum(demand)
#print(split_items)

w=[]
for item in range(number):
    given_value =length[item]
    w.extend([given_value for i in range(demand[item])])
print("Weight of split items",w)

capacity = c = L
print("Capacity of each stock (like a bin) = ",c)

UB, bin_for_item = heur_FFD(c, w)
print("UB on the number of stocks used = ",UB)
stockList = [i for i in range(UB)]

Weight of split items [5, 4, 3, 3, 3, 2, 2, 2]
Capacity of each stock (like a bin) =  8
UB on the number of stocks used =  4


In [9]:
def MODEL_1DCSP(TimeLimit,showModel):
    # Create a LP Maximization problem
    Lp_prob = p.LpProblem('csp', p.LpMinimize) 

    # Create problem Variables 
    X_vars = p.LpVariable.dicts("X",(itemList,stockList),lowBound=0,cat="Integer")
    T_vars = p.LpVariable.dicts("T",stockList,lowBound=0,cat="Integer")
    Y_vars = p.LpVariable.dicts("Y",stockList,lowBound=0,upBound=1,cat="Integer")

    # Objective Function
    Lp_prob += p.lpSum(T_vars[j] for j in stockList)

    # Constraints
    for i in itemList:
        Lp_prob += p.lpSum([X_vars[i][j] for j in stockList]) == demand[i], 'Demand('+str(i)+')'

    for j in stockList:
        Lp_prob += p.lpSum([length[i] * X_vars[i][j] for i in itemList] + T_vars[j]) == L * Y_vars[j], 'Waste('+str(j)+')'

    # Solve the problem
    status = Lp_prob.solve(p.PULP_CBC_CMD(msg=False,maxSeconds=TimeLimit))

    # Show the model
    if showModel:
        print(Lp_prob)

    # Show the solution
    print('The status of the solution:',p.LpStatus[status])
    print('The objective value =',p.value(Lp_prob.objective))
    
    # Print the solution
    OutputXList = [[X_vars[i][j].varValue for j in stockList] for i in itemList]
    print("X =",OutputXList)

    OutputTList = [T_vars[j].varValue for j in stockList]
    print("T =",OutputTList)

    OutputYList = [Y_vars[j].varValue for j in stockList]
    print("Y =",OutputYList)

    # Print the solution in words
    for i in itemList:
        for j in stockList:
            if OutputXList[i][j] > 0:
                print("Item", i, "is cut ", OutputXList[i][j], " time(s) from stock", j)

In [10]:
MODEL_1DCSP(60,False)

The status of the solution: Optimal
The objective value = 0.0
X = [[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 2.0], [2.0, 0.0, 0.0, 1.0]]
T = [0.0, 0.0, 0.0, 0.0]
Y = [1.0, 0.0, 1.0, 1.0]
Item 0 is cut  1.0  time(s) from stock 2
Item 1 is cut  1.0  time(s) from stock 0
Item 2 is cut  1.0  time(s) from stock 2
Item 2 is cut  2.0  time(s) from stock 3
Item 3 is cut  2.0  time(s) from stock 0
Item 3 is cut  1.0  time(s) from stock 3


# 1D-CSP = BPP

In [11]:
# Since the 1D-CSP is identical to the BPP, you can also solve the 1D-CSP by calling the BPP model
MODEL_BPP(60,False)

The status of the solution: Optimal
The objective value = 3.0
X = [[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]]
Y = [1.0, 1.0, 1.0, 0.0]
Item 0 is packed in bin  1
Item 1 is packed in bin  0
Item 2 is packed in bin  2
Item 3 is packed in bin  2
Item 4 is packed in bin  1
Item 5 is packed in bin  2
Item 6 is packed in bin  0
Item 7 is packed in bin  0


# The workforce assignment problem
The bin packing problem can also be used for the workforce assignment problem (but will not be further discussed here)

# The robotic process automation problem

= The robotic process automation problem is an extended version of the bin packing problem, and the Python code of this model is available as a separate jupyter notebook file.

PARAMETERS <br>
- $R$: Set of robots (index $r$)
- $T$: Set of periods (index $t$)
- $P$: Set of transaction types (index $p$)
- $v_p$: Volume of transactions of type $p$
- $t_p$: Processing time of transaction $p$ (in seconds)
- $l_t$: Length of period $t$ (in seconds)
- $w_{t,p}$: 1, if transaction type $p$ can be processed in period $t$, 0 otherwise <br>

DECISION VARIABLES <br>
- $x_{r,k,t}$ Number of transactions of type $p$ processed by robot $r$ in period $t$
- $Z_{r}$ 1, if robot $r$ is active, 0 otherwise <br>

MODEL <br>
\begin{align*}
\mathrm{Minimise} \sum_{r = 1}^{|R|} Z_{r}
\end{align*}
\begin{align}
\mathrm{s.t.}
& \sum_{t = 1}^{|T|} \sum_{r = 1}^{|R|} x_{r,t,p} = v_p  			&& p = 1, \ldots, |P| \label{RPA_volume}\\
& \sum_{r = 1}^{|R|}  x_{r,t,p} \leq w_{p,t} \times v_p  					&& p = 1, \ldots, |P|, t = 1, \ldots, |T|\label{RPA_market}\\
& \sum_{p = 1}^{|P|} t_p \times x_{r,t,p} \leq  l_{t} \times Z_p  				&& t = 1, \ldots, |T|, r = 1, \ldots, |R|\label{RPA_period}\\
& x_{r,t,p} \in \{0,1\}								&&r = 1, \ldots, |R|, t = 1, \ldots, |T|, p = 1, \ldots, |P|\label{RPA_X}\\
& Z_r \in \{0,1\}									&& r = 1, \ldots, |R|\label{RPA_Z}
\end{align}




# The 2D cutting stock problem


Cf. Separate exercise (in MS Excel available from website)